## CS336 - Lec4: Mixture of experts

## Lec4

### 引入
- 什么是MoE：将一个很大的FFN用多个很大的FFN+用于选择FFN的层代替
- 为什么MoE流行：
    1. 同样的FLOPs，更多参数效果更好
    2. 训练更快
    3. 并行
    - 缺点：infra复杂且只在多节点（超大规模）时有优势，训练目标不明确且不稳定
- MoE的应用位置：通常是代替MLP层，但也可MoE作用于attention头上

### MoE结构

#### 路由器 Routing function
- 本质：将token匹配到专家
- routing模式
    1. token选topK专家，
    2. 专家选topK token，
    3. 全局平均匹配
- 现在几乎所有MoE都是token选topK专家的路由模式
- 常用routing变体
    1. 通过线性层和softmax结果选topK
    2. hash直接分专家
    3. RL学习routes（RL适用离散决策，但是本身计算量大和不稳定）
    4. 通过定义匹配问题求解分配策略


##### topK routing
- 经典topK routing：$$h_t^l=\sum^{N}_{i=1}(g_{i,t}\text{FFN}_i(u_t^l))+u_t^l$$ 
    - $g_{i,t}=\begin{cases}s_{i,t}, & s_{i,t}\in\mathrm{TopK}(\{s_{j,t}|1\leq j\leq N\},K),\\[4pt]0, & \text{otherwise}\end{cases}$
    - $s_{i,t}=\text{Softmax}_i({u_t^l}^Te_i^l)$
    - i是专家，t是token
    - e是为每个专家学到的向量，输出可类比为专家的亲和度

##### 更多routing变体
- 传统topK routing $\to$ 细粒度专家（每个FFN的投影维度可以减小，但是总体专家数可以随之增加）Fine-grained Expert Segmentation $\to$ 共享专家 Shared Expert（不经过router）

#### 专家数量
routed数量8-256(2048)，激活数量1-8，共享专家数0-4， fine-grained ration 1/2-1/14

#### 训练目标
##### 挑战
- 矛盾核心：需要在训练的时候需要稀疏（为了效率），但是稀疏的门控不可导/argmax是阶跃函数跳变点不可导，因此梯度不能传到router，模型不知道选错专家是因为router差，还是专家没学好
- 不用argmax而用软概率训练缺陷：训练于推理分布偏移，算力成本高
- 直接训练后果：部分专家初始化好，会被一直选择，而另一部分完全学习不到，即路由坍塌（已有消融实验验证）（类似于一种局部最优）

##### 解决方案：
1. RL学习优化门控策略
2. 随机近似，即加入随机扰动（Bandit算法/多臂老虎机）
    - $$H(x)=x\cdot W_g+StandardNormal()\cdot Softplus(x\cdot W_{noise})$$
    - $$G(x)=Softmax(KeepTopK(H(x),k))$$
    - 在输入中加入随机干扰（有可学习干扰大小的矩阵），利用干扰和原始的router对专家选择的"信心"，能够在探索过程中有足够数据对所有专家进行学习

3. 加入启发式的平衡损失
    - 给定N个专家，有T个token的Batch，$$loss=\alpha N \sum_i^{N}f_iP_i$$
        - $ f_i = \frac{1}{T} \sum_{x \in \mathcal{B}} \mathbb{1}\{\text{argmax } p(x) = i\} $，是分配给专家i的token比例
        - $P_i=\frac1T\sum_{x\in \mathcal{B}}p_i(x)$，是分配给专家i的router的概率
        - $p(x)$是经过路由softmax的概率向量
        - $p_i(x)$的导数为$\frac{\alpha N}{T^2}\sum_{x \in \mathcal{B}} \mathbb{1}\{\text{argmax } p(x) = i\}$，频率越高，导数越大，会被调低权重

##### 平衡损失变体
1. 聚合口径为专家
    - $$\mathcal{L}_{\text{ExpBal}} = \alpha_1 \sum_{i=1}^{N'} f_i P_i$$
    - $f_i = \frac{N'}{K'T} \sum_{t=1}^{T} \mathbb{1}\{\text{Token } t \text{ selects Expert } i\}$
    - $P_i = \frac{1}{T} \sum_{t=1}^{T} s_{i,t}$
2. per-device 设备级均衡，因为专家很多时候是分布在不同设备上的，为了防止硬件瓶颈
    - 聚合口径为设备 $$\mathcal{L}_{\text{DevBal}} = \alpha_2 \sum_{i=1}^{D} f'_i P'_i$$
    - $f'_i = \frac{1}{|\mathcal{E}_i|} \sum_{j \in \mathcal{E}_i} f_j$
    - $P'_i = \sum_{j \in \mathcal{E}_i} P_j$
    - $\mathcal{E}_i$: 部署在设备 $i$ 上的专家集合。


##### 更多平衡损失的变体
- auxiliary loss free balancing（DeepSeek-v3）
- 在得到$s_{i,t}$后，在后面加上$b_i$（per-expert bias，作用是让专家i更可能/不可能得到token）
- $b_i$是通过在线学习得到的

#### 更多MoE训练方法
- UpCycling：将预训练后的LM里的权重和结构全部拷贝，将MLP的结构和权重复制多份，往MLP权重加上轻微扰动再进行训练

#### 使用MoE可能出现的问题
1. 直接训练路由坍塌。$\to$ 用平衡损失
2. 并行化计算上的效率问题，比如分给某个专家的Token不够，矩阵里就会有很多0填充；如果太多，多出的Token就会被丢弃。$\to$ 块稀疏矩阵以及相应的高性能库
3. 稳定性问题。$\to$ fp32+辅助z loss
4. 在微调阶段，MoE容易过拟合。$\to$ 用大量SFT数据

### DeepSeek

#### DeepSeek v1/MoE
1. 经典topK路由
2. 16B参数，2/8活跃参数
3. MoE，2个共享专家+4/64专家
4. per-Expert和per-Device辅助平衡损失

#### DeepSeek v2
1. 236B参数，21B活跃
2. 更细粒度的专家
3. 加入topM的设备路由，主要考虑设备的通信成本，作为topK的提前操作
4. 加入通信平衡损失

#### DeepSeek v3
1. 671B参数，37B活跃
2. 路由计算变为：sigmoid计算亲和度+softmax归一化topK+topM
3. 加入aux-loss-free平衡，以及sequence-wise辅助平衡损失（不是batch）

4. MLA 多头潜注意力
    - 想法：先将KV投影到更低维度，再储存，使用的时候再投影回来
    - 过程：
        1. 压缩阶段：$$c_t^{KV} = W_{DKV} h_t, \quad c_t^{Q}=W_{DQ} h_t$$
            - $c_t^{KV}$ 是仅需存储在 KV Cache 中的低维向量
        2. 引入位置编码：将 $Q$ 和 $K$ 拆分为内容部分C和位置部分R(注意不是切分，而是通过不同的矩阵计算得到，再拼接)
            - 用RoPE，$<QR_q,R_kK> = <hW^QR_q,R_kW_{UK}c_t^{KV}>$, 每次计算都与位置相关
            - 对于 Query ($q$):$$q_t = [q_t^C; q_t^R]$$其中 $q_t^C$ 是内容向量，$q_t^R$ 是带 RoPE 的位置向量。
            - 对于 Key ($k$):$$k_t = [k_t^C; k_t^R]$$其中 $k_t^R$ 也是带 RoPE 的位置向量。在 KV Cache 中，模型只需存储 $c_t^{KV}$ 和 $k_t^R$。
        3. 注意力分数计算（矩阵吸收）：$$(q_t^C)^T k_j^C = (W_{UQ} c_t^Q)^T (W_{UK} c_j^{KV})=(c_t^Q)^T W_{absorbed} c_j^{KV}$$
            - 核心结论： 在推理时，我们不需要把 $c_j^{KV}$ 还原成高维的 $k_j^C$。我们可以提前把 $W_{UQ}$ 的转置和 $W_{UK}$ 相乘，得到一个合并后的矩阵 $W_{absorbed}$。
            - 计算时，直接让当前 Token 的压缩向量 $c_t^Q$ 与缓存中的压缩向量 $c_j^{KV}$ 通过这个合并矩阵运算即可。这整个过程中，显存里存的是压缩包，计算过程也是基于压缩包完成的。
        - 完整公式 $$Score_{t,j} = \text{Softmax} \left( \frac{(c_t^Q)^T W_{absorbed} c_j^{KV} + (q_t^R)^T k_j^R}{\sqrt{d}} \right)$$ $$Output = \sum_j (Score_{t,j} \cdot W_{UV} c_j^{KV})$$

5. MTP 多token预测：简单来说就是将当前的隐藏状态embedding共享，和真实token结合，交给独立的小transformer模块预测。